# 4.3.1 Densely Connected on D2 Labels - 10 timesteps

In this notebook we will attempt to learn on the decision labels 'D2', which have a 50/50 buy/sell split without hold positions indicated. We hope this will be an easier decision function to learn, rather than the 3 class classification of buy/hold/sell. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [2]:
y = np.load('./data/prepared/august25screenfixed/numpy_matrices/yD2_60.npy')

In [3]:
y.shape

(30, 4000)

In [4]:
y_train = y[:, :2000]
y_test = y[:, 2000:]

y_train.shape, y_test.shape

((30, 2000), (30, 2000))

In [5]:
y_train = y_train.reshape([60000, 1])
y_test = y_test.reshape([60000, 1])
y_train.shape, y_test.shape

((60000, 1), (60000, 1))

In [6]:
X_train = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_train.npy')
X_test = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_test.npy')

In [7]:
X_train.shape, X_test.shape

((60000, 1, 116, 60), (60000, 1, 116, 60))

In [8]:
np.unique(y_train, return_counts=True)

(array([1., 2.]), array([38755, 21245]))

In [9]:
np.unique(y_test, return_counts=True)

(array([1., 2.]), array([33285, 26715]))

In [10]:
# Undersample to balance classes for training set ?
b = np.where(y_train == 1)[0]
s = np.where(y_train == 2)[0]

bi = np.random.choice(b, size=21000, replace=False)
si = np.random.choice(s, size=21000, replace=False)

ind = np.concatenate([bi,si])
ind.shape

(42000,)

In [11]:
np.unique(y_train[ind], return_counts=True)

(array([1., 2.]), array([21000, 21000]))

In [12]:
X_train[ind].shape

(42000, 1, 116, 60)

In [13]:
y_train = y_train[ind]
X_train = X_train[ind]

In [14]:
X_train.shape, y_train.shape

((42000, 1, 116, 60), (42000, 1))

In [15]:
X_train = X_train[:,:,:,:10]
X_train.shape

(42000, 1, 116, 10)

In [16]:
X_train.shape = (42000, 116, 10)
X_train.shape

(42000, 116, 10)

In [17]:
# decrease test size for runtime and memory concerns
b = np.where(y_test == 1)[0]
s = np.where(y_test == 2)[0]

bi = np.random.choice(b, size=20000, replace=False)
si = np.random.choice(s, size=20000, replace=False)

indt = np.concatenate([bi,si])
indt.shape

y_test = y_test[indt]
X_test = X_test[indt]
y_test.shape, X_test.shape

((40000, 1), (40000, 1, 116, 60))

In [18]:
X_test = X_test[:,:,:,:10]
X_test.shape = (40000, 116, 10)
X_test.shape

(40000, 116, 10)

In [19]:
np.unique(y_test, return_counts=True)[1]/y_test.shape[0]

array([0.5, 0.5])

In [20]:
np.unique(y_train, return_counts=True),np.unique(y_test, return_counts=True)

((array([1., 2.]), array([21000, 21000])),
 (array([1., 2.]), array([20000, 20000])))

In [21]:
y_train[np.where(y_train == 2)] = 0
y_test[np.where(y_test == 2)] = 0

np.unique(y_train, return_counts=True),np.unique(y_test, return_counts=True)

((array([0., 1.]), array([21000, 21000])),
 (array([0., 1.]), array([20000, 20000])))

#### Attempt 3 columns only

So far, the super simple feed forward network seems to perform just as well as the cnn. 

In [23]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(116, 10)),
    keras.layers.Dense(116, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/50
42000/42000 [==============================] - 7s 178us/sample - loss: 0.6713 - accuracy: 0.5780 - val_loss: 0.7191 - val_accuracy: 0.4926
Epoch 2/50
42000/42000 [==============================] - 8s 179us/sample - loss: 0.6552 - accuracy: 0.5927 - val_loss: 0.7342 - val_accuracy: 0.4838
Epoch 3/50
42000/42000 [==============================] - 7s 159us/sample - loss: 0.6501 - accuracy: 0.6050 - val_loss: 0.7210 - val_accuracy: 0.4870
Epoch 4/50
42000/42000 [==============================] - 7s 171us/sample - loss: 0.6442 - accuracy: 0.6139 - val_loss: 0.7264 - val_accuracy: 0.5095
Epoch 5/50
42000/42000 [==============================] - 7s 160us/sample - loss: 0.6430 - accuracy: 0.6112 - val_loss: 0.7210 - val_accuracy: 0.5026
Epoch 6/50
42000/42000 [==============================] - 7s 158us/sample - loss: 0.6409 - accuracy: 0.6140 - val_loss: 0.7133 - val_accuracy: 0.5141
Epoch 7/50
42000/42000 [==========================

KeyboardInterrupt: 

In [20]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(X_train)

In [27]:
predictions[4]

array([0.07750326, 0.92002517, 0.00247163], dtype=float32)

In [22]:
np.argmax(predictions)

13971

In [60]:
predictions = probability_model.predict(X_test)

In [61]:
np.argmax(predictions[0])

0

In [62]:
predictions[0]

array([9.8680556e-01, 1.2953308e-02, 2.4112873e-04], dtype=float32)

### Fewer Nodes

In [27]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(116, 10)),
    keras.layers.Dense(58, activation='relu'),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/50
42000/42000 [==============================] - 6s 149us/sample - loss: 0.6688 - accuracy: 0.5832 - val_loss: 0.7264 - val_accuracy: 0.4878
Epoch 2/50
42000/42000 [==============================] - 6s 151us/sample - loss: 0.6495 - accuracy: 0.6092 - val_loss: 0.7336 - val_accuracy: 0.4909
Epoch 3/50
42000/42000 [==============================] - 6s 146us/sample - loss: 0.6432 - accuracy: 0.6197 - val_loss: 0.7340 - val_accuracy: 0.4933
Epoch 4/50
42000/42000 [==============================] - 7s 171us/sample - loss: 0.6376 - accuracy: 0.6257 - val_loss: 0.7413 - val_accuracy: 0.4980
Epoch 5/50
42000/42000 [==============================] - 6s 147us/sample - loss: 0.6341 - accuracy: 0.6311 - val_loss: 0.7570 - val_accuracy: 0.5119
Epoch 6/50
42000/42000 [==============================] - 7s 156us/sample - loss: 0.6290 - accuracy: 0.6372 - val_loss: 0.7639 - val_accuracy: 0.5142
Epoch 7/50
42000/42000 [==========================

KeyboardInterrupt: 

In [28]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(116, 10)),
    keras.layers.Dense(29, activation='relu'),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/50
42000/42000 [==============================] - 6s 138us/sample - loss: 0.6661 - accuracy: 0.5853 - val_loss: 0.7274 - val_accuracy: 0.4893
Epoch 2/50
42000/42000 [==============================] - 6s 133us/sample - loss: 0.6446 - accuracy: 0.6191 - val_loss: 0.7379 - val_accuracy: 0.5009
Epoch 3/50
42000/42000 [==============================] - 6s 132us/sample - loss: 0.6375 - accuracy: 0.6259 - val_loss: 0.7464 - val_accuracy: 0.5038
Epoch 4/50
42000/42000 [==============================] - 6s 133us/sample - loss: 0.6356 - accuracy: 0.6299 - val_loss: 0.7515 - val_accuracy: 0.5189
Epoch 5/50
42000/42000 [==============================] - 6s 134us/sample - loss: 0.6303 - accuracy: 0.6356 - val_loss: 0.7487 - val_accuracy: 0.5102
Epoch 6/50
42000/42000 [==============================] - 6s 135us/sample - loss: 0.6307 - accuracy: 0.6346 - val_loss: 0.7564 - val_accuracy: 0.5164
Epoch 7/50
42000/42000 [==========================

#### Attempt LSTM

In [31]:
x = np.arange(24).reshape(3,4,2)
x

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [44]:
range(x.ndim)

range(0, 3)

In [45]:
np.transpose(x, axes=(0,2,1))

array([[[ 0,  2,  4,  6],
        [ 1,  3,  5,  7]],

       [[ 8, 10, 12, 14],
        [ 9, 11, 13, 15]],

       [[16, 18, 20, 22],
        [17, 19, 21, 23]]])

In [46]:
X_train.shape

(42000, 116, 10)

In [47]:
Xr_tp = np.transpose(X_train, axes=(0,2,1))
Xr_tp.shape

(42000, 10, 116)

In [48]:
Xe_tp = np.transpose(X_test, axes=(0,2,1))
Xe_tp.shape

(40000, 10, 116)

In [50]:
model = keras.Sequential([
    keras.layers.LSTM(10),
    keras.layers.Dense(2)
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/50
42000/42000 [==============================] - 85s 2ms/sample - loss: 0.6930 - accuracy: 0.5133 - val_loss: 0.6921 - val_accuracy: 0.5292
Epoch 2/50
42000/42000 [==============================] - 77s 2ms/sample - loss: 0.6891 - accuracy: 0.5335 - val_loss: 0.7100 - val_accuracy: 0.4933
Epoch 3/50
42000/42000 [==============================] - 83s 2ms/sample - loss: 0.6784 - accuracy: 0.5666 - val_loss: 0.7094 - val_accuracy: 0.4782
Epoch 4/50
42000/42000 [==============================] - 74s 2ms/sample - loss: 0.6725 - accuracy: 0.5795 - val_loss: 0.7174 - val_accuracy: 0.4882
Epoch 5/50
42000/42000 [==============================] - 79s 2ms/sample - loss: 0.6679 - accuracy: 0.5837 - val_loss: 0.7250 - val_accuracy: 0.4769
Epoch 6/50
42000/42000 [==============================] - 83s 2ms/sample - loss: 0.6648 - accuracy: 0.5882 - val_loss: 0.7400 - val_accuracy: 0.4833
Epoch 7/50
42000/42000 [==============================] 

KeyboardInterrupt: 

In [55]:
model = keras.Sequential([
    keras.layers.LSTM(2),
    keras.layers.Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/5
42000/42000 [==============================] - 67s 2ms/sample - loss: 0.6931 - accuracy: 0.5091 - val_loss: 0.6921 - val_accuracy: 0.5452
Epoch 2/5
42000/42000 [==============================] - 69s 2ms/sample - loss: 0.6928 - accuracy: 0.5088 - val_loss: 0.6923 - val_accuracy: 0.5263
Epoch 3/5
42000/42000 [==============================] - 68s 2ms/sample - loss: 0.6926 - accuracy: 0.5105 - val_loss: 0.6923 - val_accuracy: 0.5239
Epoch 4/5
42000/42000 [==============================] - 67s 2ms/sample - loss: 0.6922 - accuracy: 0.5149 - val_loss: 0.6923 - val_accuracy: 0.5156
Epoch 5/5
42000/42000 [==============================] - 71s 2ms/sample - loss: 0.6920 - accuracy: 0.5181 - val_loss: 0.6924 - val_accuracy: 0.5141


In [54]:
model = keras.Sequential([
    keras.layers.LSTM(10),
    keras.layers.Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/5
42000/42000 [==============================] - 79s 2ms/sample - loss: 0.6932 - accuracy: 0.5044 - val_loss: 0.6922 - val_accuracy: 0.4982
Epoch 2/5
42000/42000 [==============================] - 83s 2ms/sample - loss: 0.6901 - accuracy: 0.5335 - val_loss: 0.6978 - val_accuracy: 0.4897
Epoch 3/5
42000/42000 [==============================] - 82s 2ms/sample - loss: 0.6789 - accuracy: 0.5710 - val_loss: 0.7129 - val_accuracy: 0.4674
Epoch 4/5
42000/42000 [==============================] - 86s 2ms/sample - loss: 0.6691 - accuracy: 0.5854 - val_loss: 0.7152 - val_accuracy: 0.4771
Epoch 5/5
42000/42000 [==============================] - 79s 2ms/sample - loss: 0.6625 - accuracy: 0.5956 - val_loss: 0.7199 - val_accuracy: 0.4927


In [58]:
model = keras.Sequential([
    keras.layers.LSTM(1),
    keras.layers.Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/10
42000/42000 [==============================] - 74s 2ms/sample - loss: 0.6932 - accuracy: 0.4995 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
42000/42000 [==============================] - 65s 2ms/sample - loss: 0.6931 - accuracy: 0.5026 - val_loss: 0.6929 - val_accuracy: 0.5282
Epoch 3/10
42000/42000 [==============================] - 65s 2ms/sample - loss: 0.6931 - accuracy: 0.5053 - val_loss: 0.6926 - val_accuracy: 0.5529
Epoch 4/10
42000/42000 [==============================] - 67s 2ms/sample - loss: 0.6930 - accuracy: 0.5051 - val_loss: 0.6925 - val_accuracy: 0.5273
Epoch 5/10
42000/42000 [==============================] - 68s 2ms/sample - loss: 0.6930 - accuracy: 0.5081 - val_loss: 0.6922 - val_accuracy: 0.5538
Epoch 6/10
42000/42000 [==============================] - 70s 2ms/sample - loss: 0.6929 - accuracy: 0.5102 - val_loss: 0.6921 - val_accuracy: 0.5278
Epoch 7/10
42000/42000 [==============================] 

Test accuracy of .5529 doesn't sound great, but it should be noted that it is the best performance I have seen on a model thus far. On the right track? Strange though that the train accuracy is still 0.5081. Undertraining?

In [57]:
# Doesnt work 
#model = keras.Sequential([
#     keras.layers.LSTM(10),
#     keras.layers.LSTM(10),
#     keras.layers.Dense(2, activation='softmax')
    
# ])

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# model.fit(X_train, y_train, epochs=5,
#          validation_data=(X_test, y_test))

ValueError: Input 0 of layer lstm_8 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 10]

In [22]:
# X_train = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_train.npy')
# X_test = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_test.npy')

# X_train = X_train[ind]
# X_test = X_test[indt]

X_train.shape, X_test.shape

((42000, 116, 10), (40000, 116, 10))

In [20]:
X_train.shape = (42000, 116, 60)
X_test.shape = (40000, 116, 60)

In [23]:
Xr_tp = np.transpose(X_train, axes=(0,2,1))
Xe_tp = np.transpose(X_test, axes=(0,2,1))

In [24]:
Xr_tp.shape, Xe_tp.shape

((42000, 10, 116), (40000, 10, 116))

In [25]:
model = keras.Sequential([
    keras.layers.LSTM(1),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(Xr_tp, y_train, epochs=10,
         validation_data=(Xe_tp, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/10
42000/42000 [==============================] - 15s 359us/sample - loss: 0.6820 - accuracy: 0.5528 - val_loss: 0.7142 - val_accuracy: 0.5034
Epoch 2/10
42000/42000 [==============================] - 13s 317us/sample - loss: 0.6606 - accuracy: 0.6102 - val_loss: 0.7168 - val_accuracy: 0.4890
Epoch 3/10
42000/42000 [==============================] - 13s 318us/sample - loss: 0.6546 - accuracy: 0.6151 - val_loss: 0.7347 - val_accuracy: 0.4855
Epoch 4/10
42000/42000 [==============================] - 12s 293us/sample - loss: 0.6512 - accuracy: 0.6160 - val_loss: 0.7126 - val_accuracy: 0.4880
Epoch 5/10
42000/42000 [==============================] - 14s 329us/sample - loss: 0.6502 - accuracy: 0.6171 - val_loss: 0.7290 - val_accuracy: 0.4855
Epoch 6/10
42000/42000 [==============================] - 12s 288us/sample - loss: 0.6490 - accuracy: 0.6168 - val_loss: 0.7171 - val_accuracy: 0.4860
Epoch 7/10
13632/42000 [========>...........

KeyboardInterrupt: 

In [26]:
model = keras.Sequential([
    keras.layers.LSTM(10),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(Xr_tp, y_train, epochs=10,
         validation_data=(Xe_tp, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/10
42000/42000 [==============================] - 15s 361us/sample - loss: 0.6647 - accuracy: 0.5925 - val_loss: 0.7287 - val_accuracy: 0.4812
Epoch 2/10
42000/42000 [==============================] - 13s 307us/sample - loss: 0.6525 - accuracy: 0.6132 - val_loss: 0.7335 - val_accuracy: 0.4804
Epoch 3/10
42000/42000 [==============================] - 13s 305us/sample - loss: 0.6498 - accuracy: 0.6166 - val_loss: 0.7378 - val_accuracy: 0.4894
Epoch 4/10
42000/42000 [==============================] - 13s 309us/sample - loss: 0.6520 - accuracy: 0.6171 - val_loss: 0.7178 - val_accuracy: 0.4938
Epoch 5/10
42000/42000 [==============================] - 13s 309us/sample - loss: 0.6491 - accuracy: 0.6211 - val_loss: 0.7262 - val_accuracy: 0.5043
Epoch 6/10
42000/42000 [==============================] - 13s 307us/sample - loss: 0.6453 - accuracy: 0.6248 - val_loss: 0.7384 - val_accuracy: 0.5015
Epoch 7/10
42000/42000 [====================